In [ ]:
import tempfile
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml
from spf.dataset.spf_dataset import v5spfdataset

n = 1025
noise = 0.3
nthetas = 65
orbits = 4

tmpdir = tempfile.TemporaryDirectory()
tmpdirname = tmpdir.name
temp_ds_fn = f"{tmpdirname}/sample_dataset_for_ekf_n{n}_noise{noise}"

create_fake_dataset(
    filename=temp_ds_fn, yaml_config_str=fake_yaml, n=n, noise=noise, orbits=orbits
)
ds = v5spfdataset(
    temp_ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache=tmpdirname,
    paired=True,
    skip_fields=set(["signal_matrix"]),
)

In [ ]:
use_real_data = False
if use_real_data:
    # ds_fn = "/mnt/md1/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_10_03_38_21_nRX2_rx_circle.zarr"
    ds_fn = "/mnt/md1/2d_wallarray_v2_data/june_fix/wallarrayv3_2024_06_15_11_44_13_nRX2_bounce.zarr"
    precompute_cache_dir = "/home/mouse9911/precompute_cache_chunk16_sept"
else:
    ds_fn = temp_ds_fn
    precompute_cache_dir = tmpdirname
ds = v5spfdataset(
    ds_fn,
    nthetas=nthetas,
    ignore_qc=True,
    precompute_cache=precompute_cache_dir,
    paired=True,
    skip_fields=set(["signal_matrix", "windowed_beamformer"]),
)

In [ ]:
import os


output_prefix = "./" + os.path.basename(ds_fn) + "_"

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

for rx_idx in [0, 1]:
    ax[rx_idx].scatter(
        range(len(ds)),
        ds.mean_phase[f"r{rx_idx}"],
        label=f"radio{rx_idx} est phi",
        s=1.0,
        color="red",
    )
    ax[rx_idx].plot(ds.ground_truth_phis[rx_idx], label="perfect phi", color="blue")
    ax[rx_idx].plot(
        [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(len(ds))],
        label=f"radio{rx_idx} gt theta",
        color="green",
    )
    ax[rx_idx].set_title(f"Radio {rx_idx}")
    ax[rx_idx].set_xlabel("Time step")
    ax[rx_idx].set_ylabel("tehta/phi")
    ax[rx_idx].legend()
    ax[rx_idx].axhline(y=0, color="r", linestyle="-")
fig.suptitle("Phase(phi) recovered from radios after segmentation")
fig.savefig(f"{output_prefix}_raw_signal.png")

In [ ]:
from spf.filters.spf_single_radio_filter import SPFKalmanFilter


kf = SPFKalmanFilter(
    ds=ds, rx_idx=0, phi_std=5.0, p=5
)  # , phi_std=0.5, p=5, **kwargs):
kf.trajectory(max_iterations=5)[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 2, figsize=(10, 15))

for rx_idx in [0, 1]:  # [0, 1]:
    ax[1, rx_idx].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
    ax[1, rx_idx].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))

    kf = SPFKalmanFilter(
        ds=ds, rx_idx=rx_idx, phi_std=5.0, p=5, dynamic_R=True
    )  # , phi_std=0.5, p=5, **kwargs):
    trajectory = kf.trajectory(max_iterations=None, debug=True)
    jacobian = [x["jacobian"] for x in trajectory]
    zs = [x["observation"] for x in trajectory]
    # trajectory, jacobian, zs = trajectory_for_phi(rx_idx, ds)
    jacobian = np.array(jacobian)
    zs = np.array(zs)
    n = len(trajectory)
    ax[0, rx_idx].scatter(
        range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
        ds.mean_phase[f"r{rx_idx}"][:n],
        label=f"r{rx_idx} estimated phi",
        s=1.0,
        alpha=1.0,
        color="red",
    )
    ax[0, rx_idx].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
    ax[0, rx_idx].plot(jacobian, label="jacobian")
    ax[0, rx_idx].plot(zs, label="zs")
    ax[0, rx_idx].plot(np.clip(zs / jacobian, a_min=-5, a_max=5), label="zs/j")
    ax[1, rx_idx].plot(
        [ds[idx][rx_idx]["ground_truth_theta"] for idx in range(min(n, len(ds)))],
        label=f"r{rx_idx} gt theta",
    )
    reduced_gt_theta = np.array(
        [
            reduce_theta_to_positive_y(ds[idx][rx_idx]["ground_truth_theta"])
            for idx in range(min(n, len(ds)))
        ]
    )
    ax[1, rx_idx].plot(
        reduced_gt_theta,
        label=f"r{rx_idx} reduced gt theta",
    )

    xs = np.array([x["theta"] for x in trajectory])
    stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
    zscores = (xs - reduced_gt_theta) / stds

    ax[1, rx_idx].plot(xs, label="EKF-x", color="orange")
    ax[1, rx_idx].fill_between(
        np.arange(xs.shape[0]),
        xs - stds,
        xs + stds,
        label="EKF-std",
        color="orange",
        alpha=0.2,
    )

    ax[0, rx_idx].set_ylabel("radio phi")

    ax[0, rx_idx].legend()
    ax[0, rx_idx].set_title(f"Radio {rx_idx}")
    ax[1, rx_idx].legend()
    ax[1, rx_idx].set_xlabel("time step")
    ax[1, rx_idx].set_ylabel("radio theta")

    ax[2, rx_idx].hist(zscores.reshape(-1), bins=25)
fig.suptitle("Single ladies (radios) EKF")
fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
from spf.filters.spf_dualradio_filter import SPFPairedKalmanFilter


kf = SPFPairedKalmanFilter(
    ds=ds, phi_std=5.0, p=5, dynamic_R=False
)  # , phi_std=0.5, p=5, **kwargs):
trajectory = kf.trajectory(max_iterations=5, debug=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.rf import pi_norm, reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 1, figsize=(10, 15))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
kf = SPFPairedKalmanFilter(ds=ds, phi_std=5.0, p=5, dynamic_R=False)
trajectory = kf.trajectory(max_iterations=None, debug=True)

n = len(trajectory)
ax[0].scatter(
    range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
    ds.mean_phase[f"r{rx_idx}"][:n],
    label=f"r{rx_idx} estimated phi",
    s=1.0,
    alpha=1.0,
    color="red",
)
ax[0].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
ground_truth_theta = [
    pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(trajectory))
]
ax[1].plot(
    ground_truth_theta,
    label="craft gt theta",
)

xs = np.array([x["theta"] for x in trajectory])
stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
zscores = (xs - np.array(ground_truth_theta)) / stds

ax[1].plot(xs, label="EKF-x", color="orange")
ax[1].fill_between(
    np.arange(xs.shape[0]),
    xs - stds,
    xs + stds,
    label="EKF-std",
    color="orange",
    alpha=0.2,
)

ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("radio theta")

ax[2].hist(zscores.reshape(-1), bins=25)
# fig.suptitle("Single ladies (radios) EKF")
# fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

In [ ]:
from spf.filters.spf_dualradioXY_filter import SPFPairedXYKalmanFilter


kf = SPFPairedXYKalmanFilter(
    ds=ds, phi_std=5.0, p=0.001, dynamic_R=False
)  # , phi_std=0.5, p=5, **kwargs):
trajectory = kf.trajectory(max_iterations=5, debug=True, dt=0.05)
# trajectory

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from spf.filters.spf_dualradioXY_filter import SPFPairedXYKalmanFilter
from spf.rf import reduce_theta_to_positive_y

fig, ax = plt.subplots(3, 1, figsize=(10, 15))

ax[1].axhline(y=np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
ax[1].axhline(y=-np.pi / 2, ls=":", c=(0.7, 0.7, 0.7))
kf = SPFPairedXYKalmanFilter(ds=ds, phi_std=5, p=0.1, dynamic_R=True)
trajectory = kf.trajectory(
    max_iterations=1600, debug=True, noise_std=10.0, dt=1
)  # , noise_std=1)

n = len(trajectory)
ax[0].scatter(
    range(min(n, ds.mean_phase[f"r{rx_idx}"].shape[0])),
    ds.mean_phase[f"r{rx_idx}"][:n],
    label=f"r{rx_idx} estimated phi",
    s=1.0,
    alpha=1.0,
    color="red",
)
ax[0].plot(ds.ground_truth_phis[rx_idx][:n], label="perfect phi")
ground_truth_theta = [
    pi_norm(ds[idx][0]["craft_y_rad"].item()) for idx in range(len(trajectory))
]
ax[1].plot(
    ground_truth_theta,
    label="craft gt theta",
)

xs = np.array([x["theta"] for x in trajectory])
stds = np.sqrt(np.array([x["P_theta"] for x in trajectory]))
zscores = (xs - np.array(ground_truth_theta)) / stds

ax[1].plot(xs, label="EKF-x", color="orange")
ax[1].fill_between(
    np.arange(xs.shape[0]),
    xs - stds,
    xs + stds,
    label="EKF-std",
    color="orange",
    alpha=0.2,
)

ax[0].set_ylabel("radio phi")

ax[0].legend()
ax[0].set_title(f"Radio")
ax[1].legend()
ax[1].set_xlabel("time step")
ax[1].set_ylabel("radio theta")

# ax[2].hist(zscores.reshape(-1), bins=25)
# fig.suptitle("Single ladies (radios) EKF")
# fig.savefig(f"{output_prefix}_single_ladies_ekf.png")

pos_x = np.array([x["mu"][0] for x in trajectory])
pos_y = np.array([x["mu"][1] for x in trajectory])
rpos_x = np.array([x["mu"][4] for x in trajectory])
rpos_y = np.array([x["mu"][5] for x in trajectory])
vel_x = np.array([x["mu"][2] for x in trajectory])
vel_y = np.array([x["mu"][3] for x in trajectory])
gt_x = [ds[idx][0]["tx_pos_x_mm"].item() for idx in range(len(trajectory))]
gt_y = [ds[idx][0]["tx_pos_y_mm"].item() for idx in range(len(trajectory))]
ax[2].plot(gt_x, color="red")
ax[2].plot(gt_y, color="green")
ax[2].scatter(range(len(pos_x)), pos_x, color="red")
ax[2].scatter(range(len(pos_y)), pos_y, color="green")
ax[2].plot(range(len(rpos_x)), rpos_x, color="blue")
ax[2].plot(range(len(rpos_y)), rpos_y, color="black")
# ax[2].scatter(range(len(vel_x)), vel_x, color="red")
# ax[2].scatter(range(len(vel_y)), vel_y, color="green")